# 8. Typehinting

## T.O.C:

1. [General Examples](#general-examples)
2. [Typehinting Any](##-Typehinting-Any)
3. [Subtype-of vs Consistent-with](#Subtype-of-vs-Consistent-with)
4. [How to and why to typehint None](#How-&-Why-do-you-typehint-None-as-a-Default?)



This notebook aims to go over the typehinting listed in the book. Attempts at including both positive and negative examples will be made if possible, if not, only positive examples will be kept.

General comments and concepts will be left here

## Concepts

| Concept | Description |
|----------|----------|
| Gradual Typing | Is optional, does not catch type errors at runtime and does not enhance perforamnce |
| Duck Typing | "If it quacks like a duck, it is a duck" Example included |
| Nominal Typing | Type compatibility and equivalence is based on explicit declaration rather than structure/inheritance. |
| Consistent-with | Basically X is consistent with Y if X has all the methods/attributes of Y |
| Subtype-of | As it is, quite literally Cat is a subtype of Animal |

## General Examples

### Duck Typing & Nominal Typing

Since Duck and Clown here both support the `quack` method, they are considered the same type. Duck typing is only enforced at runtime and results in more flexibility, but usually at the cost of more errors.

From Wikipedia: In computer programming, duck typing is an application of the duck test—"If it walks like a duck and it quacks like a duck, then it must be a duck"—to determine whether an object can be used for a particular purpose. With nominative typing, an object is of a given type if it is declared as such (or if a type's association with the object is inferred through mechanisms such as object inheritance). With duck typing, an object is of a given type if it has all methods and properties required by that type.

In [1]:
class Bird:
    pass

class Duck(Bird):
    def quack(self):
        print('Quack!')

class Clown:
    def quack(self):
        print('Quack!')

def make_the_duck_quack(duck: Duck):
    duck.quack()

def make_the_bird_quack(bird: Bird):
    bird.quack()

The above will not pass Mypy as Bird has no attribute "quack". In the above, the Clown class is considered a duck to `make_the_duck_quack` because it has the `quack` method.

In this next portion, we create a daffy duck, while it is a duck, it will not throw an error if you pass it to `make_the_duck_quack` but it will throw an error to `make_the_bird_quack` if you run it in mypy because it is not a bird.

In [ ]:
daffy = Duck()
make_the_duck_quack(daffy)
make_the_bird_quack(daffy)

If you run this in mypy, you will get the following error:

```
error: Argument 1 to "make_the_bird_quack" has incompatible type "DaffyDuck"; expected "Bird"
```

The next example is demonstrating why you would not like Duck Typing. While duck typing allows the function to accept anything and as long as it quacks it works in runtime, it will not catch the error at compile time. This is why we use typehinting.

In [2]:
birdo = Bird()

make_the_bird_quack(birdo)

AttributeError: 'Bird' object has no attribute 'quack'

## Typehinting Any

If there is no type hint, it's automatically assumed it is any. In addition, you should be using `typing.Any` over `object`

Consider the following two functions.

`double_any` would pass mypy because Any is assumed to support every possible operation, however `double_object` would not pass mypy because `__mul__` is not defined for object.

In [3]:
from typing import Any

def double_any(x: Any) -> Any:
    return 2 * x

def double_object(x: object) -> object:
    return 2 * x

# Subtype-of vs Consistent-with

In the following example, we have `Dog`, `Cat`, `Car` which can speak and therefore in the make_speak method, they are consistent-with each other.

However in the `make_animal_speak` function, it expects an `Animal`, therefore since `Dog` and `Cat` are subtypes of `Animal` and implement the same interface as `Animal`, they can be used in the function.

In [ ]:
class Animal:
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        return "Bark"

class Cat(Animal):
    def speak(self):
        return "Meow"

class Car:
    def speak(self):
        return "Vroom"

def make_speak(obj):
    if hasattr(obj, 'speak') and callable(obj.speak):
        return obj.speak()
    else:
        raise TypeError("Object does not have a callable 'speak' method")
    
def make_animal_speak(animal: Animal):
    return animal.speak()

# How & Why do you typehint None as a Default?

For some functions, if you are expecting a string you can use '', but in some other cases you might want to use `None` instead. This includes cases where the parameter type is expected to be a mutable.

The signature you use is using Optional, where Optional[str] == Union[str, None]

In [1]:
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str]):
    pass

# In python 3.10 and beyond

def show_count_310(count: int, singular: str, plural: str | None):
    pass
    

# Concept: Avoiding returning Union types as it adds burden to the caller

Returning Union types adds burden to the caller to check the type of the return value. It is better to avoid returning Union types if possible. However, Union types are valid if the output type depends on the input type.

In [5]:
def parse_int(s: str) -> str | float:
    try:
        return float(s)
    except ValueError:
        return s

In [6]:
parse_int("123")

123.0

In [7]:
parse_int("huh")

'huh'

# Typehinting Standard/Generic Collections

A generic collection is something like a `list` where it can contain any type. However this is not very helpful as it does not tell you what the list contains. Therefore you should use `List[str]` to indicate that the list contains strings.

The usual use case of something like a `list` is that you want to apply some kind of operation that is equally applicable across all elements, therefore specifying the type makes sense.

Standard Collections are built-in collection type, things like tuples, sets, lists etc.

| Collection | Type |
|------------|------|
|list        |collections.deque|        
|set         |abc.Container|            
|frozenset   |abc.Collection|
|abc.Sequence|   abc.MutableSequence|
|abc.Set|        abc.MutableSet|

# Typehinting Tuples

This is not as "easy" as it seems. There are three common use cases

1. Tuples as records
2. Tuples as records with named fields
3. Tuples as immutable sequences

The first two are easy enough, if you require immutable storage along with a name, use named tuple.

The immutable sequence comes in when you have an indeterminate number of elements in your tuple

In [8]:
# Tuples as records

from typing import Tuple

def give_me_latlon(lat_lon: tuple[float, float]) -> str:
    return f"Latitude: {lat_lon[0]}, Longitude: {lat_lon[1]}"

In [10]:
# Tuples as records with named fields

from typing import NamedTuple

class Coordinates(NamedTuple):
    latitude: float
    longitude: float

def give_me_latlon_named(lat_lon: Coordinates) -> str:
    return f"Latitude: {lat_lon.latitude}, Longitude: {lat_lon.longitude}"

coord_example = Coordinates(37.7749, -122.4194)

# Demonstrate consistent with
give_me_latlon(coord_example)

'Latitude: 37.7749, Longitude: -122.4194'

In [13]:
# Tuples as immutable sequences

def give_me_lots_of_animals_and_legs(animal_legs: list[tuple[str, int], ...]) -> None:
    for animal, legs in animal_legs:
        print(f"{animal} has {legs} legs")

give_me_lots_of_animals_and_legs([("dog", 4), ("cat", 4), ("bird", 2)])

dog has 4 legs
cat has 4 legs
bird has 2 legs


# Local variables can be annotated too!

Within the function itself, you can annotate the local variables too!!

In [14]:
def create_a_dict() -> dict[str, int]:
    index: dict[str, int] = {}
    max_range = 5
    for i in range(max_range):
        index[str(i)] = i
    return index

In [16]:
# This is a callable!
create_a_dict()

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}